In [1]:
# import packages
# sds:2020 Docker environment
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sms
import seaborn as sns
import geopandas as gpd
import matplotlib.cm as cm
import matplotlib as mpl
import re
import os

import urllib
import json
import requests
import folium
from branca.utilities import split_six

import random
random.seed(42)    # For reproducibility
np.random.seed(42) 

# Make numeric display a bit neater
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))

In [2]:
data_fa=pd.read_csv('datafa.csv')

/opt/anaconda3/envs/sds2020/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (1,2,3,12,13,14,15,16,18,19,20,21,22,24,26,28,32,36,37,39,40,41,42,43,44,45,46,54,55,56,57,58,59,60,61,62,63,64) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data_fa.head()

,Unnamed: 0,registered_number,incorporation_date,dissolution_date,latest_accounts_cash,latest_accounts_assets,latest_accounts_liabilities,year_obs,sic_year,sic4,...,pcd2,pcds,cty,laua,ctry,rgn,ttwa,pct,lep1,lep2
0,0,00428557,NaN,NaN,nan,nan,nan,2018,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,00851482,NaN,NaN,nan,nan,nan,2018,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,02251331,NaN,NaN,nan,nan,nan,2018,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,01918723,NaN,NaN,nan,nan,nan,2018,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,01945037,NaN,NaN,nan,nan,nan,2018,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data_fa.describe()

,Unnamed: 0,latest_accounts_cash,latest_accounts_assets,latest_accounts_liabilities,year_obs,sic_year,sic4,change_sic,id,year_accounts,...,keep,merge_max_anna,max_anna,primary_uk_sic_2007,duo_direct_share,merge_fame,fame,birth_year,diss_year,age
count,"11,807,732.00","897,750.00","2,519,188.00","908,540.00","11,807,732.00","6,182,207.00","6,182,211.00","11,253,917.00","11,253,913.00","3,552,234.00",...,"5,596,972.00","11,255,553.00","11,253,917.00","2,458,550.00","2,385,823.00","11,807,732.00","11,807,732.00","10,353,312.00","5,901,414.00","10,353,312.00"
mean,"5,903,865.50","357,079.68","149,175.10","84,070.97","2,017.53","2,006.60","6,599.68",0.05,"39,653,121.63","2,016.45",...,1.00,1.99,0.50,"62,933.64",95.59,1.38,0.21,"2,006.46","2,010.17",6.85
std,"3,408,598.77","208,250,650.86","1,811,048.02","2,218,956.03",1.09,1.23,"2,084.48",0.22,"48,992,349.64",1.50,...,0.00,1.00,0.50,"19,270.92",13.83,0.75,0.41,11.36,6.99,10.53
min,0.00,"-226,421.00","-1,273,586.00",0.00,"2,009.00","1,992.00",111.00,0.00,1.00,"2,009.00",...,1.00,1.00,0.00,10.00,0.00,1.00,0.00,"1,800.00",8.00,"-1,999.00"
25%,"2,951,932.75",10.00,"3,476.00","2,310.00","2,018.00","2,007.00","4,791.00",0.00,"2,813,458.00","2,016.00",...,1.00,1.00,0.00,"47,530.00",100.00,1.00,0.00,"2,002.00","2,006.00",2.00
50%,"5,903,865.50",601.00,"17,326.00","10,199.00","2,018.00","2,007.00","6,820.00",0.00,"10,657,604.00","2,017.00",...,1.00,1.00,0.00,"64,209.00",100.00,1.00,0.00,"2,009.00","2,011.00",3.00
75%,"8,855,798.25","6,872.00","69,730.00","34,943.00","2,018.00","2,007.00","8,299.00",0.00,"71,917,388.00","2,017.00",...,1.00,3.00,1.00,"77,110.00",100.00,1.00,0.00,"2,014.00","2,015.00",8.00
max,"11,807,731.00","169,846,410,000.00","1,638,311,971.00","917,127,807.00","2,020.00","2,007.00","9,999.00",1.00,"168,733,887.00","2,020.00",...,1.00,3.00,1.00,"99,000.00",100.00,3.00,1.00,"2,018.00","2,018.00",198.00


In [5]:
data_fa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11807732 entries, 0 to 11807731
Data columns (total 65 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   Unnamed: 0                        int64  
 1   registered_number                 object 
 2   incorporation_date                object 
 3   dissolution_date                  object 
 4   latest_accounts_cash              float64
 5   latest_accounts_assets            float64
 6   latest_accounts_liabilities       float64
 7   year_obs                          int64  
 8   sic_year                          float64
 9   sic4                              float64
 10  change_sic                        float64
 11  id                                float64
 12  jurisdiction_code                 object 
 13  name                              object 
 14  normalised_name                   object 
 15  registered_addresspostal_code     object 
 16  company_type                      

In [6]:
primary_uk_sic_2007 = data_fa ["primary_uk_sic_2007"]

In [7]:
primary_uk_sic_2007.describe()

count   2,458,550.00
mean       62,933.64
std        19,270.92
min            10.00
25%        47,530.00
50%        64,209.00
75%        77,110.00
max        99,000.00
Name: primary_uk_sic_2007, dtype: float64

In [ ]:
#drop na rows to see how many dropped
data_fa['primary_uk_sic_2007'] = data_fa['primary_uk_sic_2007'].replace(np.nan, 0)
data_fa = data_fa[data_fa.primary_uk_sic_2007 != 0]

In [ ]:
data_fa.describe()

In [ ]:
#only 22% of the companies have got SIC

In [ ]:
#filter rows of SIC in primary_uk_sic_2007
#tech_SIC = data_fa[(data_fa["primary_uk_sic_2007"] == 26511) | (data_fa["primary_uk_sic_2007"] == 26513)]

In [ ]:
tech_SIC = data_fa[data_fa["primary_uk_sic_2007"].isin([26511,26513,26600,26701,32500,72190,74202,21100,21200,72110,
                                                       62020,62030,62090,63110,26301,26309,61900,26200,26110,26400,26512,
                                                       27110,27200,27310,27900,28910,71121,71122,71200,74100,74901,28110,
                                                       28120,28131,28132,28150,29100,29201,29202,29310,29320,30110,30120,
                                                       30200,30300,30400,59111,59112,59113,59120,59200,18201,18202,18203,
                                                       58210,58290,62011,62012,63120,64301,64302,64303,64304,64305,64306,
                                                       64921,64992,64999,65300])]

In [ ]:
len(tech_SIC.index)

In [ ]:
tech_SIC.to_csv('tech_SIC.csv')

In [ ]:
tech_SIC.describe() #year observed and account all 2018

In [ ]:
#filter strings using .str.contains/name.str.startswith
#count_row = data_postcode.shape[0] #len(data_postcode.index)
#count_col = data_postcode.shape[1]

In [ ]:
#data on postcode geo
data_postcode=pd.read_csv('ONSPD.csv',header = 0, usecols=['pcds','lat','long','oslaua'])

In [ ]:
data_postcode.info()

In [ ]:
data_postcode.head()

In [ ]:
#add lat long to the tech_SIC
tech_SIC_geo = pd.merge(tech_SIC, data_postcode, how="left", on=["pcds"])

In [ ]:
len(tech_SIC_geo.index)

In [ ]:
tech_SIC_geo.to_csv('tech_SIC_geo.csv')

In [ ]:
#shown on map generally

In [ ]:
gdf = gpd.GeoDataFrame(tech_SIC_geo, 
      geometry=gpd.points_from_xy(tech_SIC_geo['long'], tech_SIC_geo['lat'], crs='epsg:4326'))
gdf = gdf.to_crs('epsg:27700')

In [ ]:
gdf.plot(column='latest_accounts_assets', cmap='plasma', scheme='quantiles', k=10, markersize=1, figsize=(8,6));

In [ ]:
#groupby local authority district
gLAD = gdf.groupby(['oslaua','primary_uk_sic_2007'])['primary_uk_sic_2007'].agg(Count='count').reset_index() # oslaua companies grouped counts
gLAD.head(100)

In [ ]:
len(gLAD.index)

In [ ]:
#gLADr is for changing the format for readability only
gLADr = gLAD.pivot(index='oslaua', columns=['primary_uk_sic_2007'], values=['Count']).reset_index()
gLADr.columns = ['oslaua',26511,26513,26600,26701,32500,72190,74202,21100,21200,72110,
                                                       62020,62030,62090,63110,26301,26309,61900,26200,26110,26400,26512,
                                                       27110,27200,27310,27900,28910,71121,71122,71200,74100,74901,28110,
                                                       28120,28131,28132,28150,29100,29201,29202,29310,29320,30110,30120,
                                                       30200,30300,30400,59111,59112,59113,59120,59200,18201,18202,18203,
                                                       58210,58290,62011,62012,63120,64301,64302,64303,64304,64305,64306,
                                                       64921,64992,64999,65300]
gLADr = gLAD.replace(np.nan, 0)
gLADr.head()

In [ ]:
#search for random gLADr
gLADr.sample(3, random_state=42)

In [ ]:
#groupby local authority district
gLADt = gdf.groupby(['oslaua'])['primary_uk_sic_2007'].agg(Count='count').reset_index() # oslaua companies grouped counts
gLADt.head(100)

In [ ]:
#change coloumn name to match
cols = gLADt.columns.values
cols[0]  = 'LAD13CD'
cols[1]  = 'Counts'
gLADt.columns = cols

In [ ]:
#read LAD geojson

In [ ]:
with open('LAD.json', encoding="UTF-8") as json_file:
    LAD_json = json.load(json_file)

In [ ]:
print(gLADt["Counts"][2])
print(LAD_json["features"][0]["properties"])

In [ ]:
#would show empty in chrome but works in safari due to folium issue in showing complex large datasets
bins = [0, 12648, 16597, 20546, 24495, 28445]
m = folium.Map(location=[55, 4], zoom_start=5)
m.choropleth(
    geo_data=LAD_json,
    data=gLADt,
    columns=['LAD13CD', 'Counts'],
    key_on='feature.properties.LAD13CD',
    fill_color='YlGn',
    threshold_scale = bins,
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Rate (%)',
    highlight=True
)

In [ ]:
m

In [ ]:
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go

In [ ]:
#for some reason not loading - might need to change to px.choropleth_mapbox
fig = px.choropleth(gLADt, geojson=localauth, locations='LAD13CD', featureidkey="properties.LAD13CD", color='Counts',
                            color_continuous_scale="PurPor", labels={'label name':'label name'}, title='MAP TITLE',
                            scope="europe")
fig.show()

In [ ]:
#for some reason not loading
fig = px.choropleth(rs, geojson=rs, locations='LAD13CD', featureidkey="properties.LAD13CD", color='Counts',
                            color_continuous_scale="PurPor", labels={'label name':'label name'}, title='MAP TITLE',
                            scope="europe")
fig.show()

In [ ]:
from urllib.request import urlopen
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)



In [ ]:
with open('topo_lad.json', encoding="UTF-8") as json_file:
    topo = json.load(json_file)

In [ ]:
fig = px.choropleth_mapbox(gLADt, geojson=topo, locations='LAD13CD', color='Counts',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=3, 
                           opacity=0.5,
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
with open('LAD.geojson', encoding="UTF-8") as json_file:
    LAD_json = json.load(json_file)

In [ ]:
trace= go.Choroplethmapbox(geojson=LAD_json,
                           locations=gLADt['LAD13CD'],
                           z=gLADt['Counts'],
                           colorscale='Viridis',
                           colorbar_thickness=20,
                           hoverinfo='all',
                        )

In [ ]:
#LAD Geopackage
data = os.path.join('data','geo','localauth.json')
if not os.path.exists(data):
    localauth = gpd.read_file('https://raw.githubusercontent.com/supernovajon/tech-cluster/main/LAD.json')
    print("Downloaded localauth.json file.")
else:
    localauth = gpd.read_file(data)

In [ ]:
#add geometry to gLADt as a result set then plot
rs = pd.merge(localauth, gLADt, how="left", on=["LAD13CD"])

In [ ]:
rs

In [ ]:
rs.nlargest(10,'Counts')

In [ ]:
def plt_ldn(b):
    fig, ax = plt.subplots(1, figsize=(14, 22))
    b.plot(ax=ax, edgecolor='#cc2d2d', facecolor='None', zorder=3)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    return fig, ax

In [ ]:
fig, ax = plt_ldn(localauth)
ax.set_axis_off()
ax.set_title('UK tech number on LAD',fontsize=20, y=1.02)
rs.plot(column='Counts', ax=ax, linewidth=0, zorder=0,
        legend=True, scheme='equalinterval', edgecolor='k',
        cmap='tab20c',k=20)

In [ ]:
fig.savefig('UK tech number on LAD.png', dpi=150)#might need to add labels

In [ ]:
#examine null value

In [ ]:
data_3=pd.read_csv('OC_3.1.csv')